In [23]:
import functools

import imlib as im
import numpy as np
import pylib as py
import tensorflow as tf
import tensorflow.keras as keras
import tf2lib as tl
import tf2gan as gan
from PIL import Image
import tqdm
import matplotlib.pyplot as plt
import data
import random
import module
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt


gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    tf.config.experimental.set_virtual_device_configuration(gpus[0], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=4096)])
  except RuntimeError as e:
    print(e)


# ==============================================================================
# =                                   param                                    =
# ==============================================================================

'''py.arg('--dataset', default='summer2winter_yosemite')
py.arg('--datasets_dir', default='dataset')
py.arg('--load_size', type=int, default=256)  # load image to this size
py.arg('--crop_size', type=int, default=256)  # then crop to this size
py.arg('--batch_size', type=int, default=1)
py.arg('--epochs', type=int, default=200)
py.arg('--epoch_decay', type=int, default=100)  # epoch to start decaying learning rate
py.arg('--lr', type=float, default=0.0002)
py.arg('--beta_1', type=float, default=0.5)
py.arg('--adversarial_loss_mode', default='lsgan', choices=['gan', 'hinge_v1', 'hinge_v2', 'lsgan', 'wgan'])
py.arg('--gradient_penalty_mode', default='none', choices=['none', 'dragan', 'wgan-gp'])
py.arg('--gradient_penalty_weight', type=float, default=10.0)
py.arg('--cycle_loss_weight', type=float, default=10.0)
py.arg('--identity_loss_weight', type=float, default=0.0)
py.arg('--pool_size', type=int, default=50)  # pool size to store fake samples
args = py.args()'''
adataset='hair'
aload_size=256
acrop_size=128
abatch_size=6
apool_size=50
aadversarial_loss_mode='lsgan'
aepochs=200
agradient_penalty_weight=10.0
alr=0.0002
aepoch_decay=100
abeta_1=0.5
acycle_loss_weight=10.0
aidentity_loss_weight=0.1
agradient_penalty_mode='none'

# output_dir
output_dir = py.join('output', adataset)
py.mkdir(output_dir)

# save settings
#py.args_to_yaml(py.join(output_dir, 'settings.yml'), args)
def pointer(path):
    s=path.split('\\')[-1]
    path='data\\tests\\0.1_color\\'+s
    img = tf.io.read_file(path)
    img = tf.image.decode_png(img, 3)
    img=img.numpy()
    img = img / 255.0
    img=img*2-1
    lis=[img[0,0,0]+img[0,0,1]+img[0,0,2],img[0,0,0],img[0,0,1],img[0,0,2]]
    
    return lis
def parse_fn(path, color):
    
        ultimate.append(path)
        img = tf.io.read_file(path)
        img = tf.image.decode_png(img, 3)  # fix channels to 3
        img = tf.image.resize(img, [128, 128])
        img2 = tf.io.read_file(color)
        img2 = tf.image.decode_png(img2, 3)
        img2 = tf.image.resize(img2, [128, 128])
        img3 = tf.concat([img, img2], axis=2)
        img3=tf.cast(img3,tf.float32)
        img3 = tf.clip_by_value(img3, 0, 255) / 255.0
        img3=img3*2-1

        '''s=path.decode("utf-8").split('\\')[-1]
        img2 = tf.io.read_file('data\\test\\0.1_color\\'+s)
        img2 = tf.image.decode_png(img2, 3)
        img3=tf.concat([img,img2],axis=2)'''
        return (img3,) 
def show2(t):
    t = tf.squeeze(t)
    image=t.numpy()
    image=(image+1)/2
    image=image*255.0
    image=image.astype('uint8')
    image2 = Image.fromarray(image, mode='RGB')
    image2.show()
def imgchck(a):
    ten,=a
    ten1=ten[0,:,:,:3]
    ten2=ten[1,:,:,:3]
    show2(ten1)
    show2(ten2)
    

def createdata(l):
    n=len(l)
    lis=[]
    for k in range(n):
        s=l[k].split('\\')[-1]
        s2='data\\tests\\0.1_color\\'+s
        lis.append(s2)
    return lis
def write(txt,B2_imgs,ind=None):
    if(ind==None):
        with open(txt+'.txt', 'w') as filehandle:
            filehandle.writelines("%s\n" % place for place in B2_imgs)
    else:
        with open(txt+'.txt', 'w') as filehandle:
            filehandle.writelines("%s\n" % place[ind] for place in B2_imgs)

def read(txt):
    bcolr=[]
    with open(txt+'.txt', 'r') as filehandle:
        filecontents = filehandle.readlines()

    for line in filecontents:
        # remove linebreak which is the last character of the string
        current_place = line[:-1]

        # add item to the list
        bcolr.append(current_place)
    return bcolr
    
# ==============================================================================
# =                                    data                                    =
# ==============================================================================
colordataset=[]

B_imgs = py.glob(py.join('data\\tests', 'trainsmall'), '*.jpg')
B2_imgs=[]
for a in B_imgs:
    b=pointer(a)
    b=list(map(float, b))
    b.insert(0,a)
    B2_imgs.append(b)
pass

['data\\tests\\trainsmall\\0.jpg', -1.4156862745098038, -0.3254901960784313, -0.4666666666666667, -0.6235294117647059] <class 'str'> <class 'float'>


In [58]:

data2=sorted(B2_imgs,key=lambda x:x[2])
#write('Red\\imgs',data2,ind=0)
data3=sorted(B2_imgs,key=lambda x:x[3])
#write('Green\\imgs',data3,ind=0)
data4=sorted(B2_imgs,key=lambda x:x[4])
#write('Blue\\imgs',data4,ind=0)
data1=sorted(B2_imgs,key=lambda x:x[1])
#write('imgs',data1,ind=0)


['data\\tests\\trainsmall\\0.jpg', -1.4156862745098038, -0.3254901960784313, -0.4666666666666667, -0.6235294117647059]


In [66]:
data1=[x[0] for x in data1]
data2=[x[0] for x in data2]
data3=[x[0] for x in data3]
data4=[x[0] for x in data4]
dr1=sorted(B2_imgs,key=lambda x:x[1],reverse=True)
dr1=[x[0] for x in dr1]
dr2=sorted(B2_imgs,key=lambda x:x[2],reverse=True)
dr2=[x[0] for x in dr2]
dr3=sorted(B2_imgs,key=lambda x:x[3],reverse=True)
dr3=[x[0] for x in dr3]
dr4=sorted(B2_imgs,key=lambda x:x[4],reverse=True)
dr4=[x[0] for x in dr4]

In [99]:
length=len(dr1)
print(length)
def visualize(data1,dr1,ind):
    print(data1[ind])
    img = tf.io.read_file(data1[ind])
    t1 = tf.image.decode_png(img, 3)
    t1=tf.expand_dims(t1,axis=0)
    t1=t1.numpy()
    t1 = t1 / 255.0
    t1=t1*2-1
    s=dr1[ind].split('\\')[-1]
    path='data\\tests\\0.1_color\\'+s
    img = tf.io.read_file(path)
    t2 = tf.image.decode_png(img, 3)
    t2=tf.expand_dims(t2,axis=0)
    t2=t2.numpy()
    t2 = t2 / 255.0
    t2=t2*2-1
    img = im.immerge(np.concatenate([t1,t2], axis=0), n_rows=2)
    im.imwrite(img, py.join('output\\', 'seeing.jpg' ))
    image=Image.open('output\\seeing.jpg')
    image.show()
visualize(data1,dr1,4000)
    

4896
data\tests\trainsmall\91.jpg


In [113]:
dhyp1=data1[:1000]+data2[:1000]+data3[:1000]+data4[:1000]+data1[4000:]+data2[4000:]+data3[4000:]+data4[4000:]
dhyp2=dr1[:1000]+dr2[:1000]+dr3[:1000]+dr4[:1000]+dr1[4000:]+dr2[4000:]+dr3[4000:]+dr4[4000:]
    


data\tests\trainsmall\195.jpg


In [115]:
def parsen(path, color,p2,c2):
    
        img = tf.io.read_file(path)
        img = tf.image.decode_png(img, 3)  # fix channels to 3
        img = tf.image.resize(img, [128, 128])
        img2 = tf.io.read_file(color)
        img2 = tf.image.decode_png(img2, 3)
        img2 = tf.image.resize(img2, [128, 128])
        img3 = tf.concat([img, img2], axis=2)
        img3=tf.cast(img3,tf.float32)
        img3 = tf.clip_by_value(img3, 0, 255) / 255.0
        img3=img3*2-1
        
        img4 = tf.io.read_file(p2)
        img4 = tf.image.decode_png(img4, 3)  # fix channels to 3
        img4 = tf.image.resize(img4, [128, 128])
        img5 = tf.io.read_file(c2)
        img5 = tf.image.decode_png(img5, 3)
        img5 = tf.image.resize(img5, [128, 128])
        img6 = tf.concat([img4, img5], axis=2)
        img6=tf.cast(img6,tf.float32)
        img6 = tf.clip_by_value(img6, 0, 255) / 255.0
        img6=img6*2-1

        '''s=path.decode("utf-8").split('\\')[-1]
        img2 = tf.io.read_file('data\\test\\0.1_color\\'+s)
        img2 = tf.image.decode_png(img2, 3)
        img3=tf.concat([img,img2],axis=2)'''
        return (img3,img6) 
    
    
d1c=createdata(dhyp1)
d2c=createdata(dhyp2)
len_dataset=len(d1c)//abatch_size



dataset = tf.data.Dataset.from_tensor_slices((dhyp1,d1c,dhyp2,d2c))
dataset = dataset.shuffle(2048)
dataset = dataset.map(parsen, num_parallel_calls=tf.data.experimental.AUTOTUNE)
dataset = dataset.batch(abatch_size, drop_remainder=True)

In [116]:

A2B_pool = data.ItemPool(apool_size)
#B2A_pool = data.ItemPool(a.pool_size)
print('B',len_dataset)



session starts 

B 1264


In [117]:
# ==============================================================================
# =                                   models                                   =
# ==============================================================================

G = module.ResnetGenerator(input_shape=(acrop_size, acrop_size, 6))

D= module.ConvDiscriminator(input_shape=(acrop_size, acrop_size, 6))
#print('generator')
#print(G.summary())

#print('discriminator')
#print(D.summary())



d_loss_fn, g_loss_fn = gan.get_adversarial_losses_fn(aadversarial_loss_mode)
cycle_loss_fn = tf.losses.MeanAbsoluteError()
identity_loss_fn = tf.losses.MeanAbsoluteError()

G_lr_scheduler = module.LinearDecay(alr, aepochs * len_dataset, aepoch_decay * len_dataset)
D_lr_scheduler = module.LinearDecay(alr, aepochs * len_dataset, aepoch_decay * len_dataset)
G_optimizer = keras.optimizers.Adam(learning_rate=G_lr_scheduler, beta_1=abeta_1)
D_optimizer = keras.optimizers.Adam(learning_rate=D_lr_scheduler, beta_1=abeta_1)


# ==============================================================================
# =                                 train step                                 =
# ==============================================================================

@tf.function
def train_G(A,B):
    with tf.GradientTape() as t:

        At=A[:,:,:,3:]
        Bt=B[:,:,:,3:]
        Ai=A[:,:,:,:3]
        Bi=B[:,:,:,:3]

        A=tf.concat([Ai,Bt],axis=3)

        A2B = G(A, training=True)
        A2B=tf.concat([A2B,At],axis=3)
        A2B2A = G(A2B, training=True)
        B2B = G(B, training=True)

        A2B=tf.concat([A2B[:,:,:,:3],Bt],axis=3)
        A2B_d_logits = D(A2B, training=True)

        A2B_g_loss = g_loss_fn(A2B_d_logits)
        A2B2A_cycle_loss = cycle_loss_fn(Ai, A2B2A)
        B2B_id_loss = identity_loss_fn(Bi, B2B)

        G_loss = A2B_g_loss  + A2B2A_cycle_loss  * acycle_loss_weight   + B2B_id_loss * aidentity_loss_weight

    G_grad = t.gradient(G_loss, G.trainable_variables )
    G_optimizer.apply_gradients(zip(G_grad, G.trainable_variables ))

    return A2B, {'A2B_g_loss': A2B_g_loss,
                      'A2B2A_cycle_loss': A2B2A_cycle_loss,
                      'B2B_id_loss': B2B_id_loss}


@tf.function
def train_D(B, A2B):
    with tf.GradientTape() as t:
        B_d_logits = D(B, training=True)
        A2B_d_logits = D(A2B, training=True)

        B_d_loss, A2B_d_loss = d_loss_fn(B_d_logits, A2B_d_logits)
        #D_gp = gan.gradient_penalty(functools.partial(D, training=True), B, A2B, mode=agradient_penalty_mode)

        D_loss = B_d_loss + A2B_d_loss #+ D_gp * agradient_penalty_weight

    D_grad = t.gradient(D_loss, D.trainable_variables)
    D_optimizer.apply_gradients(zip(D_grad, D.trainable_variables ))

    return {'B_d_loss': B_d_loss + A2B_d_loss}


def train_step(A, B):
    A2B, G_loss_dict = train_G(A, B)

    # cannot autograph `A2B_pool`
    A2B = A2B_pool(A2B)  # or A2B = A2B_pool(A2B.numpy()), but it is much slower

    D_loss_dict = train_D( B, A2B)

    return G_loss_dict, D_loss_dict


@tf.function
def sample(A,B):
    At = A[0, :, :, 3:]
    At=tf.expand_dims(At, 0)
    Bt = B[0, :, :, 3:]
    Bt=tf.expand_dims(Bt, 0)
    Ai = A[0, :, :, :3]
    Ai=tf.expand_dims(Ai, 0)

    A = tf.concat([Ai, Bt], axis=3)

    A2B = G(A, training=False)
    A2B = tf.concat([A2B, At], axis=3)
    A2B2A = G(A2B, training=False)


    return Ai,Bt,A2B[:,:,:,:3], A2B2A

def show(t,strin):
    t = tf.squeeze(t)
    image=t.numpy()
    image=(image+1)/2
    image=image*255.0
    image=image.astype('uint8')
    image2 = Image.fromarray(image, mode='RGB')
    image2.save('output\\hair\\msample\\'+strin)



In [118]:
# ==============================================================================
# =                                    run                                     =
# ==============================================================================
aepochs=5

# epoch counter
ep_cnt = tf.Variable(initial_value=0, trainable=False, dtype=tf.int64)

# checkpoint
'''checkpoint = tl.Checkpoint(dict(G=G,
                                D=D,
                                G_optimizer=G_optimizer,
                                D_optimizer=D_optimizer,
                                ep_cnt=ep_cnt),
                           py.join(output_dir, 'checkpoints'),
                           max_to_keep=5)
try:  # restore checkpoint including the epoch counter
    checkpoint.restore().assert_existing_objects_matched()
except Exception as e:
    print(e)'''
# summary
#G=tf.keras.models.load_model('Models\\Generator.h5')
#D=tf.keras.models.load_model('Models\\Discriminator.h5')

train_summary_writer = tf.summary.create_file_writer(py.join(output_dir, 'summaries', 'train'))

# sample
#B_set,a=B
#test_iter = iter(B_set)
sample_dir = py.join(output_dir, 'samples_training')
py.mkdir(sample_dir)


# main loop
with train_summary_writer.as_default():
    for ep in tqdm.trange(aepochs, desc='Epoch Loop'):
        if ep < ep_cnt:
            continue
        count=0

        # update epoch counter
        ep_cnt.assign_add(1)

        # train for an epoch
        for x in tqdm.tqdm(dataset, desc='Inner Epoch Loop', total=len_dataset, position=0, leave=True):
            A,B=x
        #for A in A_set:
            G_loss_dict, D_loss_dict = train_step(A, B)

            # # summary
            tl.summary(G_loss_dict, step=G_optimizer.iterations, name='G_losses')
            tl.summary(D_loss_dict, step=G_optimizer.iterations, name='D_losses')
            tl.summary({'learning rate': G_lr_scheduler.current_learning_rate}, step=G_optimizer.iterations, name='learning rate')
            
            if G_optimizer.iterations.numpy() % 100 == 0: 
                G.save('Models\\Generator.h5')
                D.save('Models\\Discriminator.h5')
                
            # sample
            if G_optimizer.iterations.numpy() % 50 == 0: 
                '''A = next(test_iter)
                m,=A
                A=m'''
                A,B,A2B, A2B2A = sample(A, B)
                show(A2B,'iter_{}_{}.jpg'.format(ep,G_optimizer.iterations.numpy()))
                img = im.immerge(np.concatenate([A, A2B, A2B2A, B], axis=0), n_rows=2)
                im.imwrite(img, py.join(sample_dir, 'iter-%09d.jpg' % G_optimizer.iterations.numpy()))

        # save checkpoint
        #checkpoint.save(ep)

Epoch Loop: 100%|██████████| 5/5 [1:52:13<00:00, 1346.77s/it]


In [122]:
import random
model = tf.keras.models.load_model('modelsbase\\12811\\Generator.h5')

# Create a new model instance
def inp(path):
    img = tf.io.read_file(path)
    img = tf.image.decode_png(img, 3)  # fix channels to 3
    img = tf.image.resize(img, [128, 128])
    img=tf.expand_dims(img, axis=0)
    s=path.split('\\')[-1]
    path2='data\\tests\\0.1_color\\'+s
    img2 = tf.io.read_file(path2)
    img2 = tf.image.decode_png(img2, 3)  # fix channels to 3
    img2 = tf.image.resize(img2, [128, 128])
    img2=tf.expand_dims(img2, axis=0)
    img3=tf.concat([img,img2],axis=3)
    img3=tf.cast(img3,tf.float32)
    img3 = tf.clip_by_value(img3, 0, 255) / 255.0
    img3=img3*2-1
    return img3
def sample2(A,B):
    At = A[0, :, :, 3:]
    At=tf.expand_dims(At, 0)
    Bt = B[0, :, :, 3:]
    Bt=tf.expand_dims(Bt, 0)
    Ai = A[0, :, :, :3]
    Ai=tf.expand_dims(Ai, 0)

    A = tf.concat([Ai, Bt], axis=3)
    #print('never before\n',A)
    
    A2B = model(A, training=False)
    #print('ever after\n',A2B)
    A2B = tf.concat([A2B, At], axis=3)
    A2B2A = model.predict(A2B)


    return Ai,Bt,A2B[:,:,:,:3], A2B2A

# Check its architecture
devset = py.glob(py.join('data\\tests', 'dev'), '*.jpg')
color=devset.copy()
random.shuffle(color)

for i in range(100):
    A1=inp(devset[i])
    B1=inp(color[i])
    A1,B1,A2B1, A2B2A1 = sample2(A1, B1)

    #show(A2B1,'test_{}.jpg'.format(c))
    img = im.immerge(np.concatenate([A1, A2B1, A2B2A1, B1], axis=0), n_rows=2)
    im.imwrite(img, py.join('output\\test_11\\', 'test_{}.jpg'.format(i)))
print('over n out')

over n out
